# Here are all the parameters you need to set before running

1. Set params
2. Click "Run All"

## Install Related Libraries Param

In [35]:
INSTALL_RELATED_LIBRARY = False

## Migrate Mode

In [36]:
# Mode list(no need to change it)
_FROM_BIBTEX_TO_DT = 0      # Migrate from the bibtex, which manually exported from Zotero
_FROM_ZOTERO_TO_DT = 1      # Migrate from Zotero to Devonthink

In [37]:
# Set the migration mode
MODE = _FROM_ZOTERO_TO_DT

## Document Source Params

### From Zotero Params

Where To Get the Data

In [38]:
# if migrating from Zotero
if MODE == _FROM_ZOTERO_TO_DT:
    # The path to Zotero Sqlite database
    ZOTERO_DATABASE_PATH = "/Users/tftuser/Zotero/zotero.sqlite"
    # The path to Zotero storage path (we need it to locate the attachments)
    ZOTERO_STORAGE_PATH = "/Users/tftuser/Zotero/storage"
    # The library name that we want to export from Zotero. (LIBRARY_NAME has higher priority than COLLECTION_NAME, COLLECTION_NAME will work only when LIBRARY_NAME is None)
    LIBRARY_NAME = "Pamala Haynes"       # "Elyn Garret"        # "Effie Burrus"
    # Collection name that we want to export. If set None, export all collections
    COLLECTION_NAME = None      # "Effie Burrus--Michigan and Cleveland"
    # Zotero metadata tsv
    METADATA_TSV_NAME = "metadata.tsv"

Where To Save the Result
- The exported documents and DEVONthink database will be both saved in this folder

In [39]:
# if migrating from Zotero
if MODE == _FROM_ZOTERO_TO_DT:
    # All the result will be save in this folder, including DEVONthink database
    RESULT_FOLDER = "/Users/tftuser/Desktop/Migrate/Exported Items_Pamala Haynes"
    # The path to store the clean pdfs from Zotero. These pdfs are renamed, with extension. Recommended also in the RESULT_FOLDER
    RESULT_FOLDER_DOCUMENTS = "/Users/tftuser/Desktop/Migrate/Exported Items_Pamala Haynes/files"

### From Exported Files Params

- We exported documents with bib from Zotero manually using plugin

In [40]:
# If we migrate from the bibtex, which manually exported from Zotero
if MODE == _FROM_BIBTEX_TO_DT:
    # The path to the folder of unzipped exported files, "UNZIP_BASE_PATH + Exported Items_Sandy Carpenter.bib" should be able to locate the bib file
    UNZIP_BASE_PATH = "/Users/tftuser/Desktop/Migrate/Exported Items_Black Women Film Critics 316"
    # The bibtex file name
    BIB_NAME = "Exported Items_Black Women Film Critics 316.bib"     # "Exported Items_Vashti McKensie.bib"

## DEVONthink Params

In [41]:
# The DEVONthink database name
DEVONTHINK_DATABASE_NAME = "Pamala Haynes.dtBase2"

# Get Metadata from Zotero

## Use local sqlite to get data

- need to append filepath information

In [42]:
# Jupyter might show error red wave line, just ignore it. It's fine.
if INSTALL_RELATED_LIBRARY:
    %pip install pandas
    %pip install Pyarrow        # pandas required dependency
    %pip install tabulate


In [43]:
# Get the metadata of the attachments
def get_SQL_METADATA(libraryID, collection_key):
    """
    Collection might has sub collections
    """

    sql = f"""
        -- Get the attachment and its information
        SELECT
            items.key,      -- key of the attachment, also the folder name
            fields.fieldName,            -- field name
            itemDataValues.value AS fieldValue,         -- field value
            tag_info.tag,          -- tags info, duplicated for every key
            CASE
                WHEN fields.fieldName='title' THEN items.key || '/' || SUBSTR(itemAttachments.path, LENGTH('storage:') + 1)
                ELSE NULL
            END AS file        -- the file path
        FROM
            items
                -- Filter only items as attachment, which can be pdf, html, etc.
                INNER JOIN itemAttachments ON items.itemID=itemAttachments.itemID
                -- Get Metadata, using parentItemID because the info is saved on the parent not on the attachment
                LEFT JOIN itemData ON itemAttachments.parentItemID=itemData.itemID
                -- Get field name
                LEFT JOIN fields ON fields.fieldID=itemData.fieldID
                -- Get field
                LEFT JOIN itemDataValues ON itemDataValues.valueID=itemData.valueID
                -- tag
                LEFT JOIN (SELECT itemTags.itemID, GROUP_CONCAT(DISTINCT tags.name) AS tag
                           FROM itemTags LEFT JOIN tags ON itemTags.tagID=tags.tagID
                           GROUP BY itemTags.itemID
                          ) AS tag_info ON itemAttachments.parentItemID=tag_info.itemID
                -- Exclude Deleted Items (Needs deletedItems.itemID=NULL in WHERE)
                LEFT JOIN deletedItems ON items.itemID=deletedItems.itemID
        """
    
    # If needs to filter based on the library
    if (libraryID is not None) and (libraryID != ""):
        sql += "\n" + f"""
            WHERE 
                items.libraryID={libraryID}
                -- Exclude Deleted Items 
                AND deletedItems.itemID IS NULL

        """
    # If needs to filter based on the collection
    elif (collection_key is not None) and (collection_key != ""):
        sql += "\n" + f"""
            WHERE
                  -- Make sure the attachments from a specific collections
                  itemAttachments.parentItemID IN (
                      SELECT collectionItems.itemID
                      FROM collectionItems
                               LEFT JOIN collections ON collections.collectionID = collectionItems.collectionID
                      WHERE collections.key=\'{collection_key}\'
                  )
                  -- Exclude Deleted Items 
                  AND deletedItems.itemID IS NULL
            """

    return sql

In [44]:
import sys
import sqlite3
import pandas as pd


# Get metadata from the database
def get_metadata(zotero_database_path, library_name=None, collection_name=None, save_path=None):
    """
    :param save_path: save the metadata as tsv
    """
    # Get libraryID based on the library name
    def _get_libraryID(library_name, zotero_database_path):
        SQL_LIBRARY = f"""
            SELECT libraries.libraryID
            FROM libraries
                LEFT JOIN groups on libraries.libraryID = groups.libraryID
            WHERE groups.name='{library_name}'
        """

        conn = sqlite3.connect(zotero_database_path)        
        try:
            df = pd.read_sql_query(SQL_LIBRARY, conn)
        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            conn.close()

        # It should be only one result
        if len(df["libraryID"]) > 0:
            return df["libraryID"][0]
        else:
            # print(f"No Library Named: {library_name} !!!")
            return None
    
    # Get collection key based on the collection name
    def _get_collection_key(collection_name, zotero_database_path):
        SQL_COLLECTION = f"""
            SELECT collections.key
            FROM collections
            WHERE collections.collectionName=\'{collection_name}\'
        """

        conn = sqlite3.connect(zotero_database_path)        
        try:
            df = pd.read_sql_query(SQL_COLLECTION, conn)
        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            conn.close()

        # It should be only one result
        if len(df["key"]) > 0:
            return df["key"][0]
        else:
            # print(f"No Collection Named: {collection_name} !!!")
            return None


    libraryID = None
    collection_key = None
    # Get libraryID
    if library_name is not None and library_name != "":
        libraryID = _get_libraryID(library_name=library_name, zotero_database_path=zotero_database_path)
        if libraryID is None or libraryID == "":
            print(f"No Library Named: {library_name} !!!")
            sys.exit(0)
    # Get the collection key
    elif collection_name is not None and collection_name != "":
        collection_key = _get_collection_key(collection_name=collection_name, zotero_database_path=zotero_database_path)
        if collection_key is None or collection_key == "":
            print(f"No Collection Named: {collection_name} !!!")
            sys.exit(0)
    else:
        collection_key = None

    # Get metadata from Zotero
    conn = sqlite3.connect(zotero_database_path)
    try:
        # Query
        sql = get_SQL_METADATA(libraryID=libraryID, collection_key=collection_key)
        df = pd.read_sql_query(sql, conn)
        # Save the metadata content
        if save_path is not None:
            df.to_csv(save_path, sep="\t", index=False)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()
    
    return df

## Post Processing

pivot

In [45]:
import os
import numpy as np
import pandas as pd
from tabulate import tabulate       # print(tabulate(res, headers='keys', tablefmt='pretty'))


# Pivot table
def pivot(ori_df, save_path=None):
    # Function to get the first non-NaN value
    def _first_not_nan(series):
        return series.dropna().iloc[0] if not series.dropna().empty else np.nan
    
    # Pivot the table
    res_part1 = ori_df.copy()
    res_part1 = res_part1.pivot_table(index="key", columns="fieldName", values="fieldValue", aggfunc="first")   # res = res.pivot(index='key', columns='fieldName', values='fieldValue')

    # Deal with the remaining columns
    res_part2 = ori_df.copy()
    remain_cols = list(set(res_part2.columns) - set(["key", "fieldName", "fieldValue"]))     # key field is still inside
    agg_dict = {i_col: _first_not_nan for i_col in remain_cols}     # the other fields except ["keys", "fieldName", "fieldValue"]
    res_part2 = res_part2.groupby('key').agg(agg_dict).reset_index()

    # Join both parts
    res = pd.merge(res_part1, res_part2, on='key', how='left')

    # Save the result
    if save_path is not None:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)      # Create a destination fodler is not exist
        res.to_csv(save_path, sep="\t", index=False)

    return res


Fix the tsv

- drop key
- rename abstractnote into abstract

In [46]:
def fix_csv(df, save_path=None):
    # Rename key into foldername
    if "key" in df.columns:
        df = df.rename(columns={"key": "foldername"})    # df.drop("key", axis=1)
    
    # Rename abstractnote into abstract
    if "abstractNote" in df.columns:
        df = df.rename(columns={"abstractNote": "abstract"})

    # Rename tag into keywords
    if "tag" in df.columns:
        df = df.rename(columns={"tag": "keywords"})

    # Extra Citation into a seperate column
    if "extra" in df.columns and "citationKey" not in df.columns:
        df['citationkey'] = df['extra'].str.extract('Citation Key: ([^\n]*)')


    # Save the result
    if save_path is not None:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)      # Create a destination fodler is not exist
        df.to_csv(save_path, sep="\t", index=False)

    return df

## Export

- The whole thing

In [47]:
import os

def export_zotero(zotero_database_path, library_name, collection_name, save_path=None):
    res = get_metadata(zotero_database_path, library_name, collection_name, save_path=None)
    res = pivot(res, save_path=None)
    res = fix_csv(res, save_path=None)

    # Save the path
    if save_path is not None:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)      # Create a destination fodler is not exist
        res.to_csv(save_path, sep="\t", index=False) 

    return res

# Convert bib into csv

In [48]:
# Jupyter might show error red wave line, just ignore it. It's fine.
if INSTALL_RELATED_LIBRARY:
    %pip install --upgrade pip setuptools
    %pip install bibtexparser
    %pip install pybtex
    %pip install --upgrade pybtex

Solution 3:

- Use both pybtex and bibtexparser
    - Use pybtex to change the ENTRYTYPE to msci
    - Use bibtextparser to read and write the data
    - Recover the ENTRYTYPE in the csv

In [49]:
import os
import re
from pybtex.database import parse_file, Entry
from pybtex.database.output.bibtex import Writer
import bibtexparser
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import convert_to_unicode
import csv

# Change all ENTRYTYPE into misc
def _change_entrytype(bib_in_path, bib_out_path): 
    """
    # Change all ENTRYTYPE into @misc
    # IT DOESN"T WORK OUT AFTER TO_STRING()??? SO WEIRD!!!
    bib_data = parse_file(bib_in_path, bib_format="bibtex")
    for key, entry in bib_data.entries.items():
        entry.type = 'misc'
    bib_data = bib_data.to_string(bib_format="bibtex")

    writer = Writer()
    with open(bib_out_path, 'w', encoding='utf-8') as output_file:
        writer.write_stream(bib_data, output_file)
    """

    with open(bib_in_path, 'r', encoding='utf-8') as input_file:
        bib_data = input_file.read()

    # First to_string, then regex
    entrytype_pattern = re.compile(r'@\w+{')
    bib_data = re.sub(entrytype_pattern, '@misc{', bib_data)

    # To output to the file
    with open(bib_out_path, 'w', encoding='utf-8') as output_file:
        output_file.write(bib_data)

# Function to load and parse the BibTeX file
def _load_bibtex(bib_file_path):
    with open(bib_file_path, encoding='utf-8') as bibtex_file:
        parser = BibTexParser(common_strings=True)
        parser.customization = convert_to_unicode
        bib_database = bibtexparser.load(bibtex_file, parser=parser)
    return bib_database

# Function to find all unique field names in the BibTeX database
def _get_bibtex_fieldnames(bib_database):
    fieldnames = []
    for entry in bib_database.entries:
        for key in entry.keys():
            if key not in fieldnames:
                fieldnames.append(key)
    return fieldnames

# Function to write the BibTeX database to a CSV file
def _write_bibtex2csv(bib_database, csv_file_path, fieldnames):
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter='\t')
        writer.writeheader()
        for entry in bib_database.entries:
            writer.writerow({field: entry.get(field, '') for field in fieldnames})

# Main conversion function
def convert_bibtex_to_csv(bib_file_path, csv_file_path):
    _tmp_bib_path = "./tmp_bib.bib"

    _change_entrytype(bib_file_path, _tmp_bib_path)
    bib_database = _load_bibtex(_tmp_bib_path)
    
    fieldnames = _get_bibtex_fieldnames(bib_database)
    _write_bibtex2csv(bib_database, csv_file_path, fieldnames)

    # Delete the temporary bib file
    os.remove(_tmp_bib_path)


# Fix CSV and files

## File

just one file per folder (watch for ;) 
delete any extra attachments
    - what would extra attchment look like?

In [50]:
import pandas as pd


def one_file(df, col="file"):
    def _split_files(file_column_value):
        # Split using a regular expression that looks ahead for "files/"
        parts = re.split(r'(?=files/)', file_column_value)
        remove_blank = [part for part in parts if part.strip()]
        remove_semicolon = [part.strip(";") for part in remove_blank]
        return remove_semicolon

    if col not in df.columns:
        return df

    # Set as str. Otherwise the blank value might be seen as float NaN
    df[col] = df[col].astype(str)

    # Split files
    df[col] = df[col].apply(_split_files)
    # Seperate records into different rows
    df = df.explode(col)
    # Reset the index (Those splitted rows share the same index now.)
    df.reset_index(drop=True, inplace=True)

    return df



Remove the records without a file

- Not really necessary
- The blank files won't bother importing into Deonthink
- Yeah, but it doesn't hurt to make the program more robust. Devonthink might cause an error one day. Who know.

In [51]:
def remove_no_file(df, col="file"):
    if col not in df.columns:
        return df

    df = df[df[col].notna() & (df[col] != '')]
    
    return df


add extension

- Some files have no extension, append to it
- Pls make sure use both of them. Because you need to guarantee the file path and file field are consistent.
- Very special case
    + Some files without ext might have blankspaces in the end, but no blankspace in bib file path 
    + That's why we need to strip before append the extension

In [52]:
import os
import shutil

__TXT = ['txt', 'doc', 'docx', 'pdf', 'rtf', 'html', 'htm', 'xml', 'md', 'epub', 'mobi', 'azw']
__PIC = ['jpg', 'jpeg', 'png', 'gif', 'bmp', 'tiff', 'svg', 'webp']
__WEB = ['html', 'htm', 'css', 'js']
__AUDIO = ['mp3', 'wav', 'aac', 'flac', 'alac', 'ogg', 'm4a']
__VIDEO = ['mp4', 'avi', 'mov', 'wmv', 'flv', 'mkv', 'webm']
__ARCHIVE = ['zip', 'rar', '7z', 'tar', 'gz', 'bz2', 'xz']
__EXE = ['exe', 'msi', 'bin', 'sh', 'bat']
__PPT = ['ppt', 'pptx', 'odp']
__EXCEL = ['xls', 'xlsx', 'ods', 'csv']

_EXT_LIST = __TXT + __PIC + __WEB + __AUDIO + __VIDEO + __ARCHIVE + __EXE + __PPT + __EXCEL


# Add extension to records in the csv
def add_extension(df, file_in_dir, file_out_dir, col="file", ext="pdf", output=True, keep_original_file=True):
    """
        The function won't infect Zotero databse
    
        :param file_in_dir: file_in_dir + file column from df should make a full path of the pdfs
        :param file_out_dir: the copy of the file. But clean renamed. file_out_dir + file column after cleaning should make a full path of the pdfs
    """

    # Count how many files have been modified
    cnt_change = 0

    # Function to append '.pdf' if necessary
    def _append_ext(old_filepath):
        if old_filepath == "":
            new_filepath = old_filepath
        else:
            file_extension = old_filepath.split('.')[-1].lower()
            # Need to add the extension
            if file_extension not in _EXT_LIST:
                # The weird case that some files without ext might have blankspaces in the end, but no blankspace in bib file path 
                new_filepath = old_filepath.strip()
                new_filepath = new_filepath + "." + ext

                # cnt_change + 1
                cnt_change += 1
            # The old filename is already valid
            else:
                new_filepath = old_filepath

        # Rename filename in the file manager (OS)
        # Form the full path
        old_full_path = os.path.join(file_in_dir, old_filepath)
        new_full_path = os.path.join(file_out_dir, new_filepath)
        # Move
        if old_full_path != new_full_path:
            # Create a destination fodler is not exist
            os.makedirs(os.path.dirname(new_full_path), exist_ok=True)
            if keep_original_file:
                shutil.copyfile(old_full_path, new_full_path)
            else:
                os.rename(old_full_path, new_full_path)

        return new_filepath  

    # Skip when no col
    if col not in df.columns:
        if output:
            print(f"No Column {col} in the Sheet.")
        return df
    
    # Execute
    df[col] = df[col].apply(_append_ext)

    # Print
    if output:
        print(f"File Extension Added: {cnt_change} files")

    return df


Change file name to the new convention

- Old file name has ', ;, "

In [53]:
_PUNCTUATION_MAP = {
    "\'": "",
    "\"": "",
    "$": "",
    ";": " ",
    ",": " ",
    "...": "."
}


# replace the file field
def replace_punctuation_csv(df, file_in_dir, file_out_dir, col="file", keep_original_file=False):
    """
        The function won't infect Zotero databse
    
        :param file_in_dir: file_in_dir + file column from df should make a full path of the pdfs
        :param file_out_dir: the copy of the file. But clean renamed. file_out_dir + file column after cleaning should make a full path of the pdfs
    """

    # Function to apply the punctuation map to a file path
    def _clean_file_path(old_file_path, punctuation_map):
        new_file_path = old_file_path
        for key, value in punctuation_map.items():
            new_file_path = new_file_path.replace(key, value)
        
        # Rename filename in the file manager (OS)
        old_full_path = os.path.join(file_in_dir, old_file_path)
        new_full_path = os.path.join(file_out_dir, new_file_path)
        # Might change
        if old_full_path != new_full_path:
            # Create a destination fodler is not exist
            os.makedirs(os.path.dirname(new_full_path), exist_ok=True)
            # Move
            if keep_original_file:
                shutil.copyfile(old_full_path, new_full_path)
            else:
                os.rename(old_full_path, new_full_path)

        return new_file_path

    if col not in df.columns:
        return df
    
    # Apply the function to the 'file_path' column of the dataframe
    df[col] = df[col].apply(_clean_file_path, punctuation_map=_PUNCTUATION_MAP)

    return df

Make sure file column is the at the first column

In [54]:
def move_column(df, col="file", pos=0):
    if col not in df.columns:
        return df

    # Remove the column from the DataFrame
    column_to_move_data = df.pop(col)

    # Insert the column at the desired position
    df.insert(pos, col, column_to_move_data)

    return df


## @ field 

- After converting bib into csv, @ field is changed into ID, we need to convert it back.

In [55]:
def rename_ID(df, old="ID", new="@"):
    if old not in df.columns:
        return df
    
    df = df.rename(columns={old: new})

    return df

## date field

- When migrating from Zotero, the date field is like "1971-08-17 Aug 17, 1971". We need to keep just "1971-08-17"
- When migrating from Exported File, the date is perfectly "1971-08-17"

In [56]:
def keep_one_date(df, col="date"):
    if col not in df.columns:
        return df
    
    df[col] = df[col].str.strip()
    df[col] = df[col].str.split(" ").str[0]

    return df


## Keywords: replace / with -

Example: (African American-Caribbean-African, Los Angeles-Calif) is one keyword.  

Keywords should be separated by commas.  


In [57]:
def replace_seperator(df, col="keywords", old="/", new="-"):
    if col not in df.columns:
        return df
    
    df[col] = df[col].str.replace(old, new)

    return df


## Title and Abstract: replace smart quotes with ASCII

In [58]:
def replace_quotes(df, cols=["title", "abstract"]):
    QUOTE_MAP = {
        '“': '"',
        '”': '"',
        '‘': "'",
        '’': "'"
    }

    for i_col in cols:
        if i_col in df.columns:
            df[i_col] = df[i_col].replace(QUOTE_MAP, regex=True)

    return df


## Fix the comma format

- Devonthink has weird problems when reading continuous blank spaces in the row end
- So it needs one comma at the end of the row (Devonthink is weird)

In [59]:
# NOT GOOD FOR THE CSV, BUT KEEP APPLE SCRIPT NICE
# Fill enough commas for every row
def add_comma(csv_path_in, csv_path_out):
    with open(csv_path_in, 'r', encoding='utf-8') as infile, open(csv_path_out, 'w+', encoding='utf-8') as outfile:
        # Read the header and write it unchanged
        header = infile.readline()
        outfile.write(header)

        # Append a comma to the end of each subsequent line
        for line in infile:
            # If the line is not empty, append a comma
            if line.strip():
                outfile.write(line.rstrip('\n') + ',\n')
            # Write empty lines unchanged
            else:
                outfile.write(line)


In [60]:
# NOT GOOD FOR APPLE SCRIPT, BUT KEEP CSV NICE
# Remember to SKIP the last col in apple script
def add_extra_column(df):
    _USELESS_COL = "USELESS"
    _USELESS_VALUE = "USELESS_USELESS"

    df[_USELESS_COL] = _USELESS_VALUE

    return df


## Fix All

- Integrate everything

In [61]:
import pandas as pd


def fix_all(file_in_path, file_out_path, csv_path_in, csv_path_out=None):
    """
    
    :param files_in_path: 
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_path_in, sep="\t")

    # Clean the data
    # file
    df = remove_no_file(df)
    df = one_file(df)
    df = move_column(df)
    df = add_extension(df, file_in_dir=file_in_path, file_out_dir=file_out_path, col="file", ext="pdf", output=True, keep_original_file=True)
    df = replace_punctuation_csv(df, file_in_dir=file_out_path, file_out_dir=file_out_path, col="file", keep_original_file=False)   # file_in_path should be assigned file_out_path, because it's already moved by add_extension
    # keyword
    df = replace_seperator(df)
    # abstract and title
    df = replace_quotes(df)
    # Conver ID field back into @
    df = rename_ID(df, old="ID", new="@")
    # Keep only one date
    df = keep_one_date(df, col="date")
    # Fix last column problem
    df = add_extra_column(df)

    # Write the modified DataFrame back to a new CSV file
    if csv_path_out is not None:
        df.to_csv(csv_path_out, sep="\t", index=False)     # Replace the previous 

    

# Call AppleScript to Import into DEVONthink

In [62]:
import subprocess


def call_apple_script_file(script_path):
    result = subprocess.run(['osascript', script_path], capture_output=True, text=True)

    # Check the return code (0 is success)
    return result.returncode


In [63]:
import subprocess


def call_apple_script(script):
    result = subprocess.run(['osascript', "-e", script], capture_output=True, text=True)

    return result

# Run the Whole Process

## Export From Zotero

In [64]:
if MODE == _FROM_ZOTERO_TO_DT:
    export_zotero(zotero_database_path=ZOTERO_DATABASE_PATH, library_name=LIBRARY_NAME, collection_name=COLLECTION_NAME, save_path=RESULT_FOLDER+"/"+METADATA_TSV_NAME)

## Convert Bibtex to CSV

In [65]:
#
if MODE == _FROM_BIBTEX_TO_DT:
    # The Bib path
    BIB_PATH = UNZIP_BASE_PATH + "/" + BIB_NAME
    # The dirty csv path, which is converted from biblatex
    CSV_ORI_PATH = UNZIP_BASE_PATH + "/" + "csv_original.tsv"
# 
elif MODE == _FROM_ZOTERO_TO_DT:
    # The dirty csv path, which is exported from Zotero
    CSV_ORI_PATH = RESULT_FOLDER + "/" + METADATA_TSV_NAME

In [66]:
if MODE == _FROM_BIBTEX_TO_DT:
    convert_bibtex_to_csv(bib_file_path=BIB_PATH, csv_file_path=CSV_ORI_PATH)

## Fix CSV

In [67]:
# 
if MODE == _FROM_BIBTEX_TO_DT:
    # The path is "Unzipped Exported", in which "files/XXXXXXX(number)/xxx.pdf" from file field
    ORIGINAL_DOCUMENT_FOLDER_PATH = UNZIP_BASE_PATH
    # The clean document path. (if you don't want to keep the original file, then just use the same as ORIGINAL_DOCUMENT_FOLDER_PATH)
    CLEAN_DOCUMENT_FOLDER_PATH = UNZIP_BASE_PATH
    # The csv path after cleaning
    CSV_CLEAN_PATH = UNZIP_BASE_PATH + "/" + "csv_clean.tsv"
# 
elif MODE == _FROM_ZOTERO_TO_DT:
    # For Zotero, path is "zotero/storage", in which XXXXXXX(key)/xxx.pdf
    ORIGINAL_DOCUMENT_FOLDER_PATH = ZOTERO_STORAGE_PATH
    # The path to save the clean files, which are copied from Zotero database and renamed.
    CLEAN_DOCUMENT_FOLDER_PATH = RESULT_FOLDER + "/" + "files"
    # The csv path after cleaning
    CSV_CLEAN_PATH = RESULT_FOLDER + "/" + METADATA_TSV_NAME


In [68]:
# Fix the csv
if MODE == _FROM_BIBTEX_TO_DT:
    fix_all(file_in_path=ORIGINAL_DOCUMENT_FOLDER_PATH, file_out_path=CLEAN_DOCUMENT_FOLDER_PATH, csv_path_in=CSV_ORI_PATH, csv_path_out=CSV_CLEAN_PATH)
elif MODE == _FROM_ZOTERO_TO_DT:
    fix_all(file_in_path=ORIGINAL_DOCUMENT_FOLDER_PATH, file_out_path=CLEAN_DOCUMENT_FOLDER_PATH, csv_path_in=CSV_ORI_PATH, csv_path_out=CSV_CLEAN_PATH)

File Extension Added: 0 files


## Run Applescript to Migrate to Devonthink

In [69]:
#
if MODE == _FROM_BIBTEX_TO_DT:
    # The path of the pdfs (The location of the the files folder, not inside)
    DOCSPATH = CLEAN_DOCUMENT_FOLDER_PATH
    # The path of the CSV local file after cleaning
    CSVPATH_L = CSV_CLEAN_PATH
    # Database name
    DEVONTHINK_DATABASE_NAME = DEVONTHINK_DATABASE_NAME
    # The destination of the exported Davonthink.
    DB_FOLDER_PATH = UNZIP_BASE_PATH
# 
elif MODE == _FROM_ZOTERO_TO_DT:
    # The path of the pdfs (The location of the the files folder, not inside). DOCSPATH + file field should make a full path to the document
    DOCSPATH = CLEAN_DOCUMENT_FOLDER_PATH
    # The path of the CSV local file after cleaning
    CSVPATH_L = CSV_CLEAN_PATH
    # Database name
    DEVONTHINK_DATABASE_NAME = DEVONTHINK_DATABASE_NAME
    # The destination of the exported Davonthink.
    DB_FOLDER_PATH = RESULT_FOLDER

In [70]:
# This can dynamically adapt to the columns in the csv
def apple_script_import2devonthink(docspath, csvpath_l, devonthink_database_name, db_folder_path):
	script = f"""
		-- The path of the pdfs
		set DOCSPATH to "{docspath}"

		-- The path of the CSV local file after cleaning
		set CSVPATH_L to "{csvpath_l}"

		-- Database name
		set DBNAME to "{devonthink_database_name}"
		-- The destination of the exported Davonthink.
		set DB_FOLDER_PATH to "{db_folder_path}"
		set DBPATH to DB_FOLDER_PATH & "/" & DBNAME


		tell application id "DNtp"

			-- Create the new database
			try
				-- Create the new database
				set newDb to create database POSIX file DBPATH as string
				-- set current database
				set curDb to current database
				log "Database created successfully at: " & DBPATH
			on error errMsg number errorNumber
				display dialog "Failed to create database: " & errMsg
				return
			end try


			-- Import the cleaned csv
			set csv_id to import CSVPATH_L to current group
			-- Read the csv file from DevonThink (Because it's easier to handle within DavonThink.)
			set _csv_loc to location of csv_id
			set _csv_name to name of csv_id
			set _csv_loc_D to _csv_loc & _csv_name
			set csvFile to get record at (_csv_loc_D) in curDb
			-- Get the header names for the metadata names
			set csvHeaders to (columns of csvFile)
			-- Get the contents of the cells in the file
			set csvContents to (cells of csvFile)


			-- Traverse all the records in the CSV
			repeat with csvItem in csvContents

				-- Import the file, assuming the file path is in the first column
				set pdfFile to import (DOCSPATH & "/" & (item 1 of csvItem)) to current group

				-- Add the custom metadata dynamically based on the number of columns
				-- Item 1 is the file path, so we traverse from item 2
		        -- Minus one, because we add one useless column in the end
				repeat with i from 2 to ((count of csvHeaders) - 1)
					set mdKey to (item i of csvHeaders) as string
					set mdValue to (item i of csvItem)
					add custom meta data mdValue for mdKey to pdfFile
				end repeat

			end repeat

			-- Delete that csv
			delete record csv_id		-- set theRecord to search "csv_clean"

		end tell
	"""

	return script

In [71]:
script = apple_script_import2devonthink(docspath=DOCSPATH, csvpath_l=CSVPATH_L, devonthink_database_name=DEVONTHINK_DATABASE_NAME, db_folder_path=DB_FOLDER_PATH)
res = call_apple_script(script=script)

# Some tools for you to validate

## Check the number of files

- May contain some multiple files in the file column

In [72]:
if MODE == _FROM_BIBTEX_TO_DT:
    DOCUMENT_FOLDER_PATH = UNZIP_BASE_PATH + "/" + "files"
elif MODE == _FROM_ZOTERO_TO_DT:
    DOCUMENT_FOLDER_PATH = CLEAN_DOCUMENT_FOLDER_PATH

In [73]:
import os

def count_files_in_folder(folder_path):
    if not os.path.isdir(folder_path):
        print("Error: The specified path is not a directory.")
        return

    file_count = 0
    file_name = []
    for root, _, files in os.walk(folder_path):
        # Remove hidden items (Assume we won't set pdf or html as hidden files.)
        files = [s for s in files if not s.startswith('.')]


        file_count += len(files)
        if len(files) > 0:
            file_name += files


    return file_count, file_name


In [74]:
num_files, name_files = count_files_in_folder(folder_path=DOCUMENT_FOLDER_PATH)
print(f"There are {num_files} files in the folder '{DOCUMENT_FOLDER_PATH}' and its subfolders.")
print()

print("Here are the file lists")
name_files.sort(key=lambda x: x.lower())
for i_file in name_files:
    print(i_file)

There are 641 files in the folder '/Users/tftuser/Desktop/Migrate/Exported Items_Pamala Haynes/files' and its subfolders.

Here are the file lists
1968-04-08InquiringPhotographer.pdf
1968-05-04PhotoStandalone33.pdf
1968-06-11haynesPhotoStandalone18.pdf
1968-07-02haynesRubyDeeWelcomes.pdf
1968-08-03haynesHowGetShove.pdf
1968-08-17haynesNewLocalProduction.pdf
1968-10-12haynesMovieProjectionistSeized.pdf
1968-10-19PhotoStandaloneNo.pdf
1969-01-07haynesBlackBookProgram.pdf
1969-01-11haynesRubyDeeStar.pdf
1969-02-11haynesBoxOfficeMagic.pdf
1969-02-11haynesThisHistoricEra.pdf
1969-03-18grayBlackLeadersRap.pdf
1969-03-29grayCameraReview.pdf
1969-04-08haynesPlayLampoonsMilitants.pdf
1969-05-17haynesPhilaNAACPWill.pdf
1969-05-24haynesManWhoCracked.pdf
1969-06-17haynesTeenGangsTurn.pdf
1969-06-24haynesPoitierSaysWhite.pdf
1969-07-08wilderPhotoStandalone26.pdf
1969-07-26PhotoStandaloneNo.pdf
1969-09-06haynesTV1stBlackSponsored.pdf
1969-09-13TVReviewStarts.pdf
1969-09-16haynesJuliaHatersCalled.pdf

Some useful excel functions

1. Get the file name from the file path

```excel
=MID(A51, FIND("@", SUBSTITUTE(A51, "/", "@", LEN(A51)-LEN(SUBSTITUTE(A51, "/", ""))))+1, LEN(A51))
```